#Install libraries if necessary and load them

In [ ]:
!pip install beautifulsoup4
!pip install requests

In [ ]:
import requests
from bs4 import BeautifulSoup as bs

#Load the page

In [ ]:
#load page content
r = requests.get("https://books.toscrape.com/index.html")

if r.status_code == 200:
  print('Success')
elif r.status_code == 404:
  print('Not Found')

print(r.url)
print(r.headers)

soup = bs(r.text, 'html.parser')
#print(soup)
print(soup.prettify())

#Starting to scrape

In [ ]:
first_header = soup.find('h3')
#print(first_header)

headers = soup.find_all('h3')
#print(headers)

#search for a list of elements
headers_together = soup.find(['h1', 'h3'])
#print(headers_together)

headers_list = soup.find_all(['h1', 'h3'])
#print(headers_list)

#nested calls of find, find_all
body = soup.find('body')
#print(body)

div = body.find('div')
#print(div)

small = div.find('small')
#print(small)

#search for a specific string within an element using find/find_all
import re

headers = soup.find_all('h3', string=re.compile('the'))
#print(headers)

headers = soup.find_all('h3', string=re.compile('(T|t)he'))
print(headers)

[<h3><a href="catalogue/a-light-in-the-attic_1000/index.html" title="A Light in the Attic">A Light in the ...</a></h3>, <h3><a href="catalogue/tipping-the-velvet_999/index.html" title="Tipping the Velvet">Tipping the Velvet</a></h3>, <h3><a href="catalogue/the-requiem-red_995/index.html" title="The Requiem Red">The Requiem Red</a></h3>, <h3><a href="catalogue/the-dirty-little-secrets-of-getting-your-dream-job_994/index.html" title="The Dirty Little Secrets of Getting Your Dream Job">The Dirty Little Secrets ...</a></h3>, <h3><a href="catalogue/the-coming-woman-a-novel-based-on-the-life-of-the-infamous-feminist-victoria-woodhull_993/index.html" title="The Coming Woman: A Novel Based on the Life of the Infamous Feminist, Victoria Woodhull">The Coming Woman: A ...</a></h3>, <h3><a href="catalogue/the-boys-in-the-boat-nine-americans-and-their-epic-quest-for-gold-at-the-1936-berlin-olympics_992/index.html" title="The Boys in the Boat: Nine Americans and Their Epic Quest for Gold at the 1936

#CSS selector (select)

In [ ]:
class_content = soup.select('.product_price')
#print(class_content)

class_price = soup.select('.price_color')
#print(class_price)

h3_div = soup.select('div h3')
print(h3_div)

[<h3><a href="catalogue/a-light-in-the-attic_1000/index.html" title="A Light in the Attic">A Light in the ...</a></h3>, <h3><a href="catalogue/tipping-the-velvet_999/index.html" title="Tipping the Velvet">Tipping the Velvet</a></h3>, <h3><a href="catalogue/soumission_998/index.html" title="Soumission">Soumission</a></h3>, <h3><a href="catalogue/sharp-objects_997/index.html" title="Sharp Objects">Sharp Objects</a></h3>, <h3><a href="catalogue/sapiens-a-brief-history-of-humankind_996/index.html" title="Sapiens: A Brief History of Humankind">Sapiens: A Brief History ...</a></h3>, <h3><a href="catalogue/the-requiem-red_995/index.html" title="The Requiem Red">The Requiem Red</a></h3>, <h3><a href="catalogue/the-dirty-little-secrets-of-getting-your-dream-job_994/index.html" title="The Dirty Little Secrets of Getting Your Dream Job">The Dirty Little Secrets ...</a></h3>, <h3><a href="catalogue/the-coming-woman-a-novel-based-on-the-life-of-the-infamous-feminist-victoria-woodhull_993/index.html

#Scrape first page

In [ ]:
#look for all books url's and titles
book_list = soup.find_all('h3')
print(book_list[0].find('a')['href'])
print(book_list[0].find('a')['title'])


catalogue/a-light-in-the-attic_1000/index.html
A Light in the Attic


In [ ]:
#iterate through the list and get the information we neeed
for book in book_list:
  #print(book.find('a')['href'])
  #print(book.find('a')['title'])
  book_url = 'https://books.toscrape.com/' + book.find('a')['href']
  #print(book_url)
  book_soup = bs(requests.get(book_url).text, 'html.parser')
  title = book_soup.find('h1').string
  #print(title)
  price = float(book_soup.find('p', class_='price_color').string[2:])
  #print(price)
  rating = book_soup.find('p', class_='star-rating')['class'][1]
  #print(rating)
  print(f'{title}, {price}, {rating}')
  category = book_soup.find('ul', class_='breadcrumb').find_all('a')[2].string
  #print(category)


A Light in the Attic, 51.77, Three
Tipping the Velvet, 53.74, One
Soumission, 50.1, One
Sharp Objects, 47.82, Four
Sapiens: A Brief History of Humankind, 54.23, Five
The Requiem Red, 22.65, One
The Dirty Little Secrets of Getting Your Dream Job, 33.34, Four
The Coming Woman: A Novel Based on the Life of the Infamous Feminist, Victoria Woodhull, 17.93, Three
The Boys in the Boat: Nine Americans and Their Epic Quest for Gold at the 1936 Berlin Olympics, 22.6, Four
The Black Maria, 52.15, One
Starving Hearts (Triangular Trade Trilogy, #1), 13.99, Two
Shakespeare's Sonnets, 20.66, Four
Set Me Free, 17.46, Five
Scott Pilgrim's Precious Little Life (Scott Pilgrim #1), 52.29, Five
Rip it Up and Start Again, 35.02, Five
Our Band Could Be Your Life: Scenes from the American Indie Underground, 1981-1991, 57.25, Three
Olio, 23.88, One
Mesaerion: The Best Science Fiction Stories 1800-1849, 37.59, One
Libertarianism for Beginners, 51.33, Two
It's Only the Himalayas, 45.17, Two


#Get info from all 50 pages

In [ ]:
title_list = []
price_list = []
rating_list = []
category_list = []
url_list = []

for i in range(1,51):
  url = f'https://books.toscrape.com/catalogue/page-{i}.html'
  soup = bs(requests.get(url).text, 'html.parser')
  allbooks = soup.find_all('article',class_='product_pod')

  for book in allbooks:
    book_url = "https://books.toscrape.com"
    title = book.find('h3').find('a')['title']
    price = float(book.find('p', class_='price_color').string[2:])
    #print(title)
    #print(price)

    stock = book.find('p', class_='instock availability').text.strip()
    #print(stock)

    rating = book.find('p', class_='star-rating')['class'][1]
    #print(rating)

    link = book_url + book.find('h3').find('a')['href']
    #print(link)

    title_list.append(title)
    price_list.append(price)
    rating_list.append(rating)
    url_list.append(link)



In [ ]:
import pandas as pd
df_all = pd.DataFrame({'Title': title_list, 'Price': price_list, 'Rating': rating_list, 'URL': url_list})
df_all.head()

,Title,Price,Rating,URL
0,A Light in the Attic,51.77,Three,https://books.toscrape.coma-light-in-the-attic...
1,Tipping the Velvet,53.74,One,https://books.toscrape.comtipping-the-velvet_9...
2,Soumission,50.10,One,https://books.toscrape.comsoumission_998/index...
3,Sharp Objects,47.82,Four,https://books.toscrape.comsharp-objects_997/in...
4,Sapiens: A Brief History of Humankind,54.23,Five,https://books.toscrape.comsapiens-a-brief-hist...
